In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd

#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc


In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 0
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
cleaned = False
if cleaned == True:
    path = os.getcwd()+'/'
else:
    path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'_cleaned.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

In [19]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(float)


c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)

#Normalise the features
c_fingerprints = (c_fingerprints - np.mean(c_fingerprints,axis=0))/np.std(c_fingerprints,axis=0)

fingerprints = np.concatenate((p_fingerprints,c_fingerprints),axis=1)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = fingerprints.shape
labels = labels[1:]

In [20]:
print p_fingerprints.shape
print c_fingerprints.shape

(3494, 112)
(3494, 33)


In [22]:
if(cleaned == False):
    p_fingerprints[(p_fingerprints==0)] = -1
    labels2 = np.zeros((len(labels),1))
    for i,l in enumerate(labels):
        if l=='Active':
            labels2[i] = 1
        else:
            labels2[i] = 0
else:
    labels2 = np.asarray(labels)


labels2 = labels2.astype(float)
labels2 = labels2.astype(int)

total_pos = np.sum(labels2)
print("tot_positive",total_pos)


('tot_positive', 162)


In [23]:
no_examples = fingerprints.shape[0]
ind = np.arange(no_examples)
np.random.shuffle(ind)
fingerprints = fingerprints[ind]
labels2 = labels2[ind]

In [24]:
print fingerprints.shape

(3494, 145)


In [25]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        #print("train_ind ",s_ind1,end_ind1,s_ind2,end_ind2)
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        p_train_data = fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_train_data[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(float)
    else:
        None
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [26]:
def get_val_data(validation_iter,binary = True):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    if(binary==True):
        train_data = fingerprints[s_ind:e_ind]
    else:
        None
    labels_val = labels2[s_ind:e_ind]   
    #print("val ind ",s_ind,e_ind)
    return Variable(torch.cuda.FloatTensor(train_data)),labels_val  
        

In [27]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,500)
        self.l2 = nn.Linear(500,500)
        self.l3 = nn.Linear(500,250)
        self.l4 = nn.Linear(250,50)
        self.l5 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [28]:
min_fn = 15
max_fp = 160
cm_list = []
batch_size = 128
for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(5,50,10)
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(3000):
            train_batch,target = get_train_batch(batch_size,binary = True,validation_iter = val_iter)
            model_op = mymlp(train_batch)
            #print(model_op.type)
            #print(target.type)
            loss = criterion(model_op,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter,binary = True)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        if(fn < 3):
            cm_list.append(wcf)
            if(fn<4):
                if(fp < max_fp):
                    max_fp = fp
                    model_path = os.getcwd() + '/model_all_feat_file' + fname
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([659,   0,  39,   0]))
('tn, fp, fn, tp: ', array([659,   0,  39,   0]))
('tn, fp, fn, tp: ', array([652,   7,  37,   2]))
('tn, fp, fn, tp: ', array([222, 437,   7,  32]))
('tn, fp, fn, tp: ', array([478, 181,  24,  15]))
('tn, fp, fn, tp: ', array([106, 553,   0,  39]))
('tn, fp, fn, tp: ', array([391, 268,  19,  20]))
('tn, fp, fn, tp: ', array([411, 248,  20,  19]))
('tn, fp, fn, tp: ', array([122, 537,   0,  39]))
('tn, fp, fn, tp: ', array([454, 205,  24,  15]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([666,   0,  32,   1]))
('tn, fp, fn, tp: ', array([658,   8,  28,   5]))
('tn, fp, fn, tp: ', array([610,  56,  22,  11]))
('tn, fp, fn, tp: ', array([618,  48,  25,   8]))
('tn, fp, fn, tp: ', array([473, 193,  14,  19]))
('tn, fp, fn, tp: ', array([501, 165,  16,  17]))
('tn, fp, fn, tp: ', array([ 33, 633,   0,  33]))
('tn, fp, fn, tp: ', array([ 29, 637,   0,  33]))
('tn, fp, fn, tp: ', array([ 25, 641,   0,  33]))
('tn, fp, fn, 

In [45]:
p_fingerprints_test = []
c_fingerprints_test = []
labels_test = []
path = os.getcwd() + '/bioassay-datasets/'
with open(path+fname+'red_test.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints_test.append(row[:bf])
        c_fingerprints_test.append(row[bf:-1])
        labels_test.append(row[-1])
        
p_fingerprints_test = np.asarray(p_fingerprints_test)[1:]
p_fingerprints_test = p_fingerprints_test.astype(int)
p_fingerprints_test[(p_fingerprints_test==0)] = -1

c_fingerprints_test = np.asarray(c_fingerprints_test)[1:]
c_fingerprints_test = c_fingerprints_test.astype(float)
print c_fingerprints_test.shape
#Normalise the features
c_fingerprints_test = (c_fingerprints_test - np.mean(c_fingerprints_test,axis=0))/np.std(c_fingerprints_test,axis=0)

fingerprints_test = np.concatenate((p_fingerprints_test,c_fingerprints_test),axis=1)

#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples_test , ip_dim_test) = fingerprints_test.shape
labels_test = labels_test[1:]

(856, 32)


In [44]:
print ip_dim_test,ip_dim
print p_fingerprints_test.shape, c_fingerprints_test.shape

144 145
(856, 112) (856, 32)


In [34]:
labels2_test = np.zeros((len(labels_test),1))
for i,l in enumerate(labels_test):
    if l=='Active':
        labels2_test[i] = 1
    else:
        labels2_test[i] = 0
labels2_test = labels2_test.astype(int)
total_pos = np.sum(labels2_test)
print("tot_positive",total_pos)

('tot_positive', 12)


In [36]:
test_model = c_mlp().cuda()
test_x = Variable(torch.cuda.FloatTensor(fingerprints_test).cuda())
#model_path = os.getcwd() + '/model_all_feat_file'+fname
#test_model.load_state_dict(torch.load(model_path))
test_op = test_model(test_x)

RuntimeError: size mismatch at /pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:243

In [ ]:
test_op = test_op.cpu().data.numpy()
pred_labels = np.argmax(test_op,axis=1)
cf = metrics.confusion_matrix(labels2_test,pred_labels).ravel()
print('tn, fp, fn, tp: ',cf)